In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import joblib
import re
import sys
sys.path.insert(0, '../')
import numpy as np

# Datasets
from aif360.datasets import MEPSDataset19
# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler



c:\Users\srinivas\anaconda3\Lib\site-packages\torch\_functorch\deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
dataset_orig_panel19_train = MEPSDataset19()

In [3]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

In [4]:
dataset_orig_panel19_train.feature_names[1]

'RACE'

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Prepare data
X = dataset_orig_panel19_train.features
y = dataset_orig_panel19_train.labels.ravel()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = X_train
X_test_scaled = X_test

# Train logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Predict probabilities on the same scaled training data
train_probabilities = model.predict_proba(X_train_scaled)[:, 1]

# Calculation of discrimination index without modifying dataset structure
sens_attr_index = dataset_orig_panel19_train.feature_names.index('RACE')



# Define unprivileged and privileged values
unprivileged_val = 0.0
privileged_val = 1.0



c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Discrimination Loss Fn

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Custom loss function
def discrimination_loss(output, target, sensitive_features, lambda_val=100, k=2):
    criterion = nn.BCELoss()
    standard_loss = criterion(output, target)
    mask_unpriv = (sensitive_features == 0)
    mask_priv = (sensitive_features == 1)
    #discrimination=torch.abs(prob_priv)
    prob_unpriv = torch.mean(output[mask_unpriv])
    prob_priv = torch.mean(output[mask_priv])
    discrimination = lambda_val * (prob_priv - prob_unpriv) ** k
    loss_val=(1 + lambda_val * discrimination) * standard_loss
    return loss_val,discrimination.item() 

def calculate_accuracy(predictions, targets):
    predicted_classes = (predictions >= 0.5).float()
    return (predicted_classes == targets).float().mean()


data = torch.tensor(X_train_scaled).float()
targets = torch.tensor(y_train).float().unsqueeze(1)

# Correctly preparing the sensitive features
threshold = 0.5  # Adjust the threshold according to your specific case
sensitive_features = torch.tensor((data[:, 1].numpy() > threshold).astype(float)).float()
features = torch.cat((data[:, :1], data[:, 2:]), dim=1)

# Assuming similar preparation for test data
test_data = torch.tensor(X_test_scaled).float()
test_targets = torch.tensor(y_test).float().unsqueeze(1)
test_sensitive_features = torch.tensor((test_data[:, 1].numpy() > threshold).astype(float)).float()
test_features = torch.cat((test_data[:, :1], test_data[:, 2:]), dim=1)

model2 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model2.parameters(), lr=0.01)
train_losses, train_accuracies, train_discriminations,train_fairness = [], [], [],[]
test_losses, test_accuracies, test_discriminations,test_fairness = [], [], [],[]

# Training loop
model2.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model2(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model2.eval()
    with torch.no_grad():
        test_outputs = model2(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses.append(loss.item())
    train_accuracies.append(train_accuracy.item() * 100)
    train_discriminations.append(discrimination)
    fairness=1-discrimination
    train_fairness.append(fairness)
    test_fairness.append(fairness)
    test_losses.append(test_loss.item())
    test_accuracies.append(test_accuracy.item() * 100)
    test_discriminations.append(test_discrimination)
    model2.train()


Epoch 1, Train Loss: 2.673090934753418, Train Acc: 39.25%, Train Discrimination: 0.022887492552399635 Test Loss: 61.42726135253906, Test Acc: 58.78%,Test Discrimination: 0.7591049075126648
Epoch 2, Train Loss: 38.965476989746094, Train Acc: 58.88%, Train Discrimination: 0.483476847410202 Test Loss: 33.263851165771484, Test Acc: 81.62%,Test Discrimination: 0.3896600008010864
Epoch 3, Train Loss: 20.107635498046875, Train Acc: 82.00%, Train Discrimination: 0.23562787473201752 Test Loss: 15.587469100952148, Test Acc: 82.66%,Test Discrimination: 0.17575567960739136
Epoch 4, Train Loss: 9.372156143188477, Train Acc: 82.87%, Train Discrimination: 0.10379645973443985 Test Loss: 7.601461887359619, Test Acc: 82.66%,Test Discrimination: 0.07924606651067734
Epoch 5, Train Loss: 4.696783542633057, Train Acc: 82.87%, Train Discrimination: 0.04625870659947395 Test Loss: 3.873684883117676, Test Acc: 82.66%,Test Discrimination: 0.034578826278448105
Epoch 6, Train Loss: 2.5706000328063965, Train Acc: 8

In [31]:
import plotly.graph_objects as go

def plot_metric(title, y_label, train_data, test_data):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=train_data, mode='lines+markers',
        name='Train',
        line=dict(color='RoyalBlue', width=2),
        marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Adding Test Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=test_data, mode='lines+markers',
        name='Test',
        line=dict(color='Crimson', width=2, dash='dot'),
        marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

# Example usage
plot_metric("Training and Testing Loss", "Loss", train_losses, test_losses)
plot_metric("Training and Testing Accuracy", "Accuracy (%)", train_accuracies, test_accuracies)
plot_metric("Training and Testing Discrimination", "Discrimination", train_discriminations, test_discriminations)
plot_metric("Training and Testing Fairness", "Fairness", train_fairness, test_fairness)



## BCE Loss Fn

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Custom loss function|
def discrimination_loss(output, target, sensitive_features, lambda_val=100, k=2):
    criterion = nn.BCELoss()
    standard_loss = criterion(output, target)

    mask_unpriv = (sensitive_features == 0)
    mask_priv = (sensitive_features == 1)
    #discrimination=torch.abs(prob_priv)
    prob_unpriv = torch.mean(output[mask_unpriv])
    prob_priv = torch.mean(output[mask_priv])
    discrimination = lambda_val*(prob_priv - prob_unpriv) ** k

# Handle cases where one group might be missing
    #discrimination=torch.abs(prob_priv)
 
    loss_val=standard_loss

    return loss_val,discrimination.item() 

def calculate_accuracy(predictions, targets):
    predicted_classes = (predictions >= 0.5).float()
    return (predicted_classes == targets).float().mean()


data = torch.tensor(X_train_scaled).float()
targets = torch.tensor(y_train).float().unsqueeze(1)

# Correctly preparing the sensitive features
threshold = 0.5  # Adjust the threshold according to your specific case
sensitive_features = torch.tensor((data[:, 1].numpy() > threshold).astype(float)).float()
features = torch.cat((data[:, :1], data[:, 2:]), dim=1)

# Assuming similar preparation for test data
test_data = torch.tensor(X_test_scaled).float()
test_targets = torch.tensor(y_test).float().unsqueeze(1)
test_sensitive_features = torch.tensor((test_data[:, 1].numpy() > threshold).astype(float)).float()
test_features = torch.cat((test_data[:, :1], test_data[:, 2:]), dim=1)

model = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.01)
train_losses1, train_accuracies1, train_discriminations1,train_fairness1 = [], [], [],[]
test_losses1, test_accuracies1, test_discriminations1,test_fairness1 = [], [], [],[]
# Training loop
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses1.append(loss.item())
    train_accuracies1.append(train_accuracy.item() * 100)
    train_discriminations1.append(discrimination)
    fairness=1-discrimination
    train_fairness1.append(fairness)
    test_fairness1.append(fairness)
    test_losses1.append(test_loss.item())
    test_accuracies1.append(test_accuracy.item() * 100)
    test_discriminations1.append(test_discrimination)
    model.train()


Epoch 1, Train Loss: 0.6893765330314636, Train Acc: 56.96%, Train Discrimination: 0.028892265632748604 Test Loss: 0.8665273189544678, Test Acc: 82.66%,Test Discrimination: 0.11413952708244324
Epoch 2, Train Loss: 0.8494628667831421, Train Acc: 82.87%, Train Discrimination: 0.0646490603685379 Test Loss: 0.7340521812438965, Test Acc: 82.66%,Test Discrimination: 0.02978677488863468
Epoch 3, Train Loss: 0.7180451154708862, Train Acc: 82.87%, Train Discrimination: 0.01662645861506462 Test Loss: 0.49305757880210876, Test Acc: 82.66%,Test Discrimination: 0.0008751878049224615
Epoch 4, Train Loss: 0.4822317659854889, Train Acc: 82.87%, Train Discrimination: 0.0016992390155792236 Test Loss: 0.40948766469955444, Test Acc: 84.37%,Test Discrimination: 0.5070899724960327
Epoch 5, Train Loss: 0.4052659273147583, Train Acc: 84.97%, Train Discrimination: 0.41889819502830505 Test Loss: 0.5344403386116028, Test Acc: 71.70%,Test Discrimination: 1.4629510641098022
Epoch 6, Train Loss: 0.5349717736244202, 

In [33]:
import plotly.graph_objects as go

def plot_metric(title, y_label, train_data, test_data):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=train_data, mode='lines+markers',
        name='Train',
        line=dict(color='RoyalBlue', width=2),
        marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Adding Test Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=test_data, mode='lines+markers',
        name='Test',
        line=dict(color='Crimson', width=2, dash='dot'),
        marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

# Example usage
plot_metric("Training and Testing Loss", "Loss", train_losses1, test_losses1)
plot_metric("Training and Testing Accuracy", "Accuracy (%)", train_accuracies1, test_accuracies1)
plot_metric("Training and Testing Discrimination", "Discrimination", train_discriminations1, test_discriminations1)
plot_metric("Training and Testing Fairness", "Fairness", train_fairness1, test_fairness1)



In [34]:
plot_metric("Disc Aware Model and Unaware Model Fairness", "Fairness", test_fairness, test_fairness1)
